# Sémantiques de Classement et Argumentation Probabiliste

**Navigation**: [← Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-8-Agent-Dialogues →](Tweety-8-Agent-Dialogues.ipynb)

---

## Objectifs pédagogiques

1. Comprendre les sémantiques de classement (ranking) des arguments
2. Comparer différentes approches: Categorizer, Burden-Based, Discussion-Based, Tuples*, SAF-Based
3. Découvrir l'argumentation probabiliste avec distributions sur sous-graphes
4. Maîtriser les loteries d'argumentation et les fonctions d'utilité

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


## Partie 5b : Classement et Probabilités

Cette section explore des méthodes d'analyse qui vont au-delà de l'acceptation binaire, en établissant des **classements** entre arguments ou en intégrant des **probabilités** dans le raisonnement argumentatif.

### 5.7 Sémantiques Basées sur le Classement (Ranking)

Au lieu de simplement déterminer si un argument est accepté ou rejeté, les sémantiques de classement établissent un **ordre** entre les arguments, du plus "fort" au moins acceptable.

Principales approches implémentées dans Tweety:
- **Categorizer**: Propagation de valeurs
- **Burden-Based / Discussion-Based**: Comptage attaquants/défenseurs
- **Tuples***: Matrices de valeurs (Cayrol & Lagasquie-Schiex)
- **Strategy-Based**: Basé sur les jeux (Matt & Toni)
- **SAF-Based**: Approche sociale
- **Counting / Propagation**: Sémantiques itératives

In [2]:
# --- 5.7 Sémantiques Basées sur le Classement (Ranking) ---
print("\n--- 5.7 Semantiques Basees sur le Classement (Ranking) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution des exemples de Ranking Semantics...")
    ranking_imports_ok = False
    try:
        import jpype
        from jpype.types import *

        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.rankings.reasoner import (
             CategorizerRankingReasoner, BurdenBasedRankingReasoner, DiscussionBasedRankingReasoner,
             TuplesRankingReasoner, StrategyBasedRankingReasoner, SAFRankingReasoner,
             CountingRankingReasoner, PropagationRankingReasoner
        )
        from org.tweetyproject.arg.rankings.util import RankingTools
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.math.probability import Probability

        print("Imports pour Ranking Semantics reussis.")
        ranking_imports_ok = True

        if ranking_imports_ok:
            print("\nDefinition des AAFs exemples...")

            # Example 1 (Bonzon et al. AAAI 2016)
            aaf_ex1 = DungTheory()
            args1 = {name: Argument(name) for name in "abcde"}
            for arg in args1.values(): aaf_ex1.add(arg)
            attacks1 = [("a","e"), ("d","a"), ("e","d"), ("c","e"), ("b","c"), ("b","a")]
            for s, t in attacks1: aaf_ex1.add(Attack(args1[s], args1[t]))
            print("   - AAF Example 1 (Bonzon) defini.")

            # Example 2 (Pu et al. CoRR 2015)
            aaf_ex2 = DungTheory()
            args2 = {f"x{i}": Argument(f"x{i}") for i in range(1, 5)}
            for arg in args2.values(): aaf_ex2.add(arg)
            attacks2 = [("x2","x3"), ("x2","x1"), ("x3","x2"), ("x3","x3"), ("x4","x2")]
            for s, t in attacks2: aaf_ex2.add(Attack(args2[s], args2[t]))
            print("   - AAF Example 2 (Pu) defini.")

            # Example 5 (Delobelle Thesis 2017)
            aaf_ex5 = DungTheory()
            args5 = {name: Argument(name) for name in "abcdefghij"}
            for arg in args5.values(): aaf_ex5.add(arg)
            attacks5 = [("a","b"), ("b","c"), ("b","f"), ("d","g"), ("d","f"),
                        ("e","h"), ("e","d"), ("e","i"), ("h","g"), ("j","i")]
            for s, t in attacks5: aaf_ex5.add(Attack(args5[s], args5[t]))
            print("   - AAF Example 5 (Delobelle) defini.")

            # Example 4a (Matt & Toni JELIA 2008)
            aaf_ex4a = DungTheory()
            args4a = {name: Argument(name) for name in "abcdefg"}
            for arg in args4a.values(): aaf_ex4a.add(arg)
            attacks4a = [("b","a"),("c","a"),("d","a"),("f","a"),("e","d"),("g","f")]
            for s, t in attacks4a: aaf_ex4a.add(Attack(args4a[s], args4a[t]))
            print("   - AAF Example 4a (Matt & Toni) defini.")

            print("\n--- Application des Raisonneurs de Classement ---")

            # 1. Categorizer
            print("\n* Categorizer:")
            try:
                r_cat = CategorizerRankingReasoner()
                rank_cat1 = r_cat.getModel(aaf_ex1)
                rank_cat2 = r_cat.getModel(aaf_ex2)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_cat1, 2)}")
                print(f"  - AAF Ex2: {RankingTools.roundRanking(rank_cat2, 3)}")
            except Exception as e: print(f"   Erreur Categorizer: {e}")

            # 2. Burden-Based
            print("\n* Burden-Based:")
            try:
                r_bb = BurdenBasedRankingReasoner()
                rank_bb1 = r_bb.getModel(aaf_ex1)
                print(f"  - AAF Ex1: {rank_bb1}")
            except Exception as e: print(f"   Erreur BurdenBased: {e}")

            # 3. Discussion-Based
            print("\n* Discussion-Based:")
            try:
                r_db = DiscussionBasedRankingReasoner()
                rank_db1 = r_db.getModel(aaf_ex1)
                print(f"  - AAF Ex1: {rank_db1}")
            except Exception as e: print(f"   Erreur DiscussionBased: {e}")

            # 4. Tuples*
            print("\n* Tuples*:")
            try:
                r_tup = TuplesRankingReasoner()
                rank_tup5 = r_tup.getModel(aaf_ex5)
                print(f"  - AAF Ex5: {rank_tup5}")
            except Exception as e: print(f"   Erreur Tuples: {e}")

            # 5. Strategy-Based
            print("\n* Strategy-Based:")
            try:
                r_sb = StrategyBasedRankingReasoner()
                rank_sb4a = r_sb.getModel(aaf_ex4a)
                print(f"  - AAF Ex4a: {RankingTools.roundRanking(rank_sb4a, 3)}")
            except Exception as e: print(f"   Erreur StrategyBased: {e}")

            # 6. SAF-Based
            print("\n* SAF-Based (SimpleProduct):")
            try:
                r_saf = SAFRankingReasoner()
                rank_saf1 = r_saf.getModel(aaf_ex1)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_saf1, 2)}")
            except Exception as e: print(f"   Erreur SAFBased: {e}")

            # 7. Counting Semantics
            print("\n* Counting Semantics:")
            try:
                r_count = CountingRankingReasoner(0.98, 0.001)
                rank_count1 = r_count.getModel(aaf_ex1)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_count1, 2)}")
            except Exception as e: print(f"   Erreur Counting: {e}")

            # 8. Propagation
            print("\n* Propagation Semantics:")
            try:
                r_prop1 = PropagationRankingReasoner(0.75, False, PropagationRankingReasoner.PropagationSemantics.PROPAGATION1)
                rank_prop1_ex5 = r_prop1.getModel(aaf_ex5)
                print(f"  - AAF Ex5 (e=0.75): {rank_prop1_ex5}")
            except Exception as e: print(f"   Erreur Propagation: {e}")

    except ImportError as e: print(f"Erreur d'import pour Ranking Semantics : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale Ranking: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue Ranking: {e_gen}"); import traceback; traceback.print_exc()


--- 5.7 Semantiques Basees sur le Classement (Ranking) ---
JVM prete. Execution des exemples de Ranking Semantics...
Imports pour Ranking Semantics reussis.

Definition des AAFs exemples...
   - AAF Example 1 (Bonzon) defini.
   - AAF Example 2 (Pu) defini.
   - AAF Example 5 (Delobelle) defini.
   - AAF Example 4a (Matt & Toni) defini.

--- Application des Raisonneurs de Classement ---

* Categorizer:
  - AAF Ex1: {a=0.38, b=1.0, c=0.5, d=0.65, e=0.53}
  - AAF Ex2: {x1=0.716, x2=0.397, x3=0.521, x4=1.0}

* Burden-Based:
  - AAF Ex1: [b > d > c > e > a]

* Discussion-Based:
  - AAF Ex1: [b > d > c > e > a]

* Tuples*:
  - AAF Ex5: [j = e = a > g = f > c > h = d = b > i]

* Strategy-Based:
  - AAF Ex4a: {a=0.167, b=1.0, c=1.0, d=0.25, e=1.0, f=0.25, g=1.0}

* SAF-Based (SimpleProduct):
  - AAF Ex1: {a=0.17, b=0.48, c=0.25, d=0.33, e=0.3}

* Counting Semantics:
  - AAF Ex1: {a=0.18, b=1.0, c=0.51, d=0.68, e=0.66}

* Propagation Semantics:
  - AAF Ex5 (e=0.75): [j = e = a > c > h = d = b

### 5.8 Argumentation Probabiliste

L'argumentation probabiliste introduit l'**incertitude** dans les cadres d'argumentation:

1. **Distributions sur sous-graphes**: Le graphe lui-même est incertain
2. **Acceptabilité probabiliste**: Probabilité qu'un argument soit accepté
3. **Loteries d'argumentation**: Décision sous incertitude avec utilités

Tweety implémente:
- `SubgraphProbabilityFunction`: Distribution sur les sous-graphes possibles
- `getAcceptanceProbability(arg, semantics)`: Probabilité d'acceptation
- `Division`: Partition IN/OUT/UNDEC
- `ArgumentationLottery`, `UtilityFunction`: Décision et utilité attendue

In [3]:
# --- 5.8 Argumentation Probabiliste ---
print("\n--- 5.8 Argumentation Probabiliste ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple d'argumentation probabiliste...")
    prob_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from java.util import Collection, List as JavaList, Set as JavaSet

        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import AbstractExtensionReasoner, SimpleGroundedReasoner
        from org.tweetyproject.arg.dung.semantics import Extension, Semantics
        from org.tweetyproject.arg.dung.divisions import Division

        from org.tweetyproject.arg.prob.lotteries import SubgraphProbabilityFunction, ArgumentationLottery, UtilityFunction
        from org.tweetyproject.math.probability import Probability

        print("Imports pour Argumentation Probabiliste reussis.")
        prob_imports_ok = True

        if prob_imports_ok:
            print("\n1. Creation de l'AAF de base...")
            theory_prob = DungTheory()
            a = Argument("a"); b = Argument("b"); c = Argument("c")
            theory_prob.add(a); theory_prob.add(b); theory_prob.add(c)
            theory_prob.add(Attack(a, b)); theory_prob.add(Attack(b, a)); theory_prob.add(Attack(c, b))
            print("   AAF: ", theory_prob)

            print("\n2. Extensions Grounded (pour reference)...")
            reasoner_gr = SimpleGroundedReasoner()
            extensions_gr_coll = reasoner_gr.getModels(theory_prob)
            print("   Extensions Grounded:", extensions_gr_coll)

            print("\n3. Creation d'une fonction de probabilite uniforme sur les sous-graphes...")
            prob_function = SubgraphProbabilityFunction(theory_prob)
            print(f"   Nombre de sous-graphes possibles: {prob_function.size()}")

            print("\n4. Calcul des probabilites d'acceptation (Semantique Grounded)...")
            target_semantics = Semantics.GROUNDED_SEMANTICS
            try:
                prob_obj_a = prob_function.getAcceptanceProbability(a, target_semantics)
                prob_obj_b = prob_function.getAcceptanceProbability(b, target_semantics)
                prob_obj_c = prob_function.getAcceptanceProbability(c, target_semantics)
                val_a = prob_obj_a.doubleValue(); val_b = prob_obj_b.doubleValue(); val_c = prob_obj_c.doubleValue()
                print(f"   P(Accepte({a})) = {val_a:.4f}")
                print(f"   P(Accepte({b})) = {val_b:.4f}")
                print(f"   P(Accepte({c})) = {val_c:.4f}")

                print("\n   Calcul probabilite pour Division ({a, c}, {b})...")
                in_set = Extension(); in_set.add(a); in_set.add(c)
                out_set = Extension(); out_set.add(b)
                example_division = Division(in_set, out_set)
                prob_obj_div = prob_function.getAcceptanceProbability(example_division, target_semantics)
                val_div = prob_obj_div.doubleValue()
                print(f"   P(Division={example_division}) = {val_div:.4f}")

            except jpype.JException as e_prob_calc_java: print(f"   Erreur Java calcul probabilites: {e_prob_calc_java.message()}")
            except Exception as e_prob_calc: print(f"   Erreur calcul probabilites: {e_prob_calc}")

            print("\n5. Loteries d'Argumentation et Utilite Attendue...")
            try:
                standard_divisions_coll = Division.getStandardDivisions(theory_prob)
                print(f"   Divisions standard: {standard_divisions_coll}")

                if not standard_divisions_coll.isEmpty():
                    lottery = ArgumentationLottery(standard_divisions_coll, prob_function, target_semantics)
                    print("\n   Loterie basee sur les divisions standard:")
                    print(f"     {lottery}")

                    utility_func = UtilityFunction()
                    from java.util import ArrayList
                    standard_divisions_list = ArrayList(standard_divisions_coll)

                    Division_class = jpype.JClass("org.tweetyproject.arg.dung.divisions.Division")
                    if standard_divisions_list.size() >= 1: utility_func.put(JObject(standard_divisions_list.get(0), Division_class), 10.0)
                    if standard_divisions_list.size() >= 2: utility_func.put(JObject(standard_divisions_list.get(1), Division_class), -5.0)

                    if not utility_func.isEmpty():
                        print("\n   Fonction d'utilite exemple:")
                        print(f"     {utility_func}")
                        expected_utility = utility_func.getExpectedUtility(lottery)
                        print(f"\n   Utilite Attendue de la loterie: {expected_utility.doubleValue():.4f}")

            except jpype.JException as e_lottery_java: print(f"   Erreur Java (Loteries/Utilite): {e_lottery_java.message()}")
            except Exception as e_lottery_py: print(f"   Erreur Python (Loteries/Utilite): {e_lottery_py}")

    except ImportError as e: print(f"Erreur d'import Prob: {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale Prob: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue Prob: {e_gen}"); import traceback; traceback.print_exc()


--- 5.8 Argumentation Probabiliste ---
JVM prete. Execution de l'exemple d'argumentation probabiliste...
Imports pour Argumentation Probabiliste reussis.

1. Creation de l'AAF de base...
   AAF:  <{ a, b, c },[(c,b), (a,b), (b,a)]>

2. Extensions Grounded (pour reference)...
   Extensions Grounded: [{a,c}]

3. Creation d'une fonction de probabilite uniforme sur les sous-graphes...
   Nombre de sous-graphes possibles: 19

4. Calcul des probabilites d'acceptation (Semantique Grounded)...
   P(Accepte(a)) = 0.5263
   P(Accepte(b)) = 0.3158
   P(Accepte(c)) = 0.6316

   Calcul probabilite pour Division ({a, c}, {b})...
   P(Division=({a,c}, {b})) = 0.3158

5. Loteries d'Argumentation et Utilite Attendue...
   Divisions standard: [({a,b}, {c}), ({c}, {a,b}), ({b}, {a,c}), ({a}, {b,c}), ({}, {a,b,c}), ({a,b,c}, {}), ({b,c}, {a}), ({a,c}, {b})]

   Loterie basee sur les divisions standard:
     [ 0.05263157894736842,({a,b}, {c}) ; 0.15789473684210525,({c}, {a,b}) ; 0.10526315789473684,({b}, 

---

## Résumé

Ce notebook a couvert deux extensions importantes à l'argumentation abstraite:

### Sémantiques de Classement

| Sémantique | Principe | Résultat |
|------------|----------|----------|
| **Categorizer** | Propagation de valeurs numériques | Scores dans [0, 1] |
| **Burden-Based** | Comptage attaquants/défenseurs | Ordre total ou partiel |
| **Discussion-Based** | Longueur des discussions gagnantes | Ordre total ou partiel |
| **Tuples*** | Matrices d'acceptabilité (Cayrol) | Équivalences + ordre |
| **Strategy-Based** | Jeux d'argumentation (Matt & Toni) | Scores probabilistes |
| **SAF-Based** | Votes sociaux (SimpleProduct) | Scores agrégés |
| **Counting** | Itérations de comptage | Scores convergents |
| **Propagation** | Influence des voisins | Ordre avec epsilon |

### Argumentation Probabiliste

- **SubgraphProbabilityFunction**: Distribution sur les 2^n sous-graphes possibles
- **Acceptabilité probabiliste**: P(arg ∈ extension | sous-graphe)
- **Division**: Partition (IN, OUT) représentant un état possible
- **Loterie**: Distribution sur les divisions avec utilités attendues

### Points clés

1. Les sémantiques de **classement** permettent de nuancer au-delà de l'acceptation binaire
2. Le **Categorizer** et **Counting** convergent vers des valeurs numériques interprétables
3. **Burden-Based** et **Discussion-Based** produisent des ordres comparables
4. L'**argumentation probabiliste** modélise l'incertitude sur le graphe lui-même
5. Les **loteries** permettent la prise de décision rationnelle sous incertitude argumentative

## Prochaines étapes

Le notebook suivant explore les **dialogues argumentatifs multi-agents** avec le module `agents.dialogues`.

---

**Navigation**: [← Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-8-Agent-Dialogues →](Tweety-8-Agent-Dialogues.ipynb)